In [ ]:
import os
import torch
import numpy as np
torch_version = str(torch.__version__)
scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
!pip install torch-scatter -f $scatter_src
!pip install torch-sparse -f $sparse_src
!pip install torch-geometric

import torch_sparse, torch_scatter
import pandas as pd

In [2]:
!pip install decoupler omnipath anndata

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.2.2 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/COLAB/CollecTRI")

Mounted at /content/drive


In [4]:
################################## CollecTRI ##################################
# https://github.com/saezlab/CollecTRI

# processed regulons
import decoupler as dc
net = dc.get_collectri(organism='human', split_complexes=False, load_meta=True)


  File "/usr/local/lib/python3.10/dist-packages/decoupler/omnip.py", line 587, in get_collectri
    ct = op.interactions.CollecTRI.get(
  File "/usr/local/lib/python3.10/dist-packages/omnipath/_core/requests/_utils.py", line 114, in wrapper
    return wrapped(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/omnipath/_core/requests/_utils.py", line 31, in _get_helper
    return cls()._get(**kwargs)
  File "/usr/local/lib/python3.10/dist-packages/omnipath/_core/requests/_request.py", line 108, in _get
    kwargs = self._validate_params(kwargs)
  File "/usr/local/lib/python3.10/dist-packages/omnipath/_core/requests/_request.py", line 197, in _validate_params
    res[self._query_type(k).param] = self._query_type(k)(v)
  File "/usr/local/lib/python3.10/dist-packages/omnipath/_core/query/_query.py", line 136, in __call__
    return self.value(value)
  File "/usr/local/lib/python3.10/dist-packages/omnipath/constants/_constants.py", line 51, in __call__
    return super().__cal

  0%|          | 0.00/2.43M [00:00<?, ?B/s]

0.00B [00:00, ?B/s]

In [5]:
# use protein sequence as input features
feature_name = "protein_embeddings_ProtEnc"
feature = pd.read_csv('/content/drive/MyDrive/COLAB/GENERAL_DATA/PROTEIN/protein_embeddings_ProtEnc.csv',index_col=0)


In [6]:
df = net.copy()
df.columns = ["x_name","y_name", 'weight', 'PMID']

# feature for x side
common_elements = list(set(df['x_name']) & set(feature.index))
df = df[df['x_name'].isin(common_elements)]
feature_x = feature[feature.index.isin(common_elements)]

## feature for y side
common_elements = list(set(df['y_name']) & set(feature.index))
df = df[df['y_name'].isin(common_elements)]
feature_y = feature[feature.index.isin(common_elements)]


# Create a mapping from unique x indices to range [0, num_x_nodes):
unique_x_id = df['x_name'].unique()
unique_x_id = pd.DataFrame(data={
    'x_name': unique_x_id,
    'mappedID': pd.RangeIndex(len(unique_x_id)),
})
# convert feature to tensor
feature_x = feature_x.loc[unique_x_id['x_name'], :]
feature_x = feature_x.reset_index()
feature_x = feature_x.drop_duplicates(subset='gene')
feature_x.index = feature_x.gene
feature_x.drop(columns=['gene'], inplace=True)
feature_x = feature_x.loc[:, (feature_x != 0).any(axis=0)]
feature_x = torch.from_numpy(feature_x.values).float()

# Create a mapping from unique y indices to range [0, num_y_nodes):
unique_y_id = df['y_name'].unique()
unique_y_id = pd.DataFrame(data={
    'y_name': unique_y_id,
    'mappedID_y': pd.RangeIndex(len(unique_y_id)),
})
# convert feature to tensor
feature_y = feature_y.loc[unique_y_id['y_name'], :]
feature_y = feature_y.reset_index()
feature_y = feature_y.drop_duplicates(subset='gene')
feature_y.index = feature_y.gene
feature_y.drop(columns=['gene'], inplace=True)
feature_y = feature_y.loc[:, (feature_y != 0).any(axis=0)]
feature_y = torch.from_numpy(feature_y.values).float()


# Perform merge to obtain the edges from x and y:
df_x_name = pd.merge(df['x_name'], unique_x_id,on='x_name', how='left')
df_x_name = torch.from_numpy(df_x_name['mappedID'].values)
df_y_name = pd.merge(df['y_name'], unique_y_id,on='y_name', how='left')
df_y_name = torch.from_numpy(df_y_name['mappedID_y'].values)

# With this, we are ready to construct our `edge_index` in COO format
# following PyG semantics:
edge_index_x_to_y = torch.stack([df_x_name, df_y_name], dim=0)

print()
print("Final edge indices pointing from x to y:")
print("=================================================")
print(edge_index_x_to_y)



Final edge indices pointing from x to y:
tensor([[   0,    1,    2,  ...,   38,  320,   78],
        [   0,    1,    1,  ..., 4360,   39, 4361]])


In [184]:
df_x_name.shape
df_y_name.shape
edge_index_x_to_y.shape

torch.Size([27106])

In [7]:
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

data = HeteroData()

# Save node indices:
data["x_name"].node_id = torch.arange(len(unique_x_id))
data["y_name"].node_id = torch.arange(len(unique_y_id))

triplet_names = ("x_name", "relation", "y_name")
triplet_names_rev = ("y_name", "rev_relation", "x_name")

In [8]:
# Add the node features and edge indices:
# The node feature could be added as appropriate !!!!!!!!!!!!
data["x_name"].x = feature_x
data["y_name"].x = feature_y
data[triplet_names].edge_index = edge_index_x_to_y


data = T.ToUndirected()(data)

print(data)

HeteroData(
  x_name={
    node_id=[961],
    x=[961, 480],
  },
  y_name={
    node_id=[4362],
    x=[4362, 480],
  },
  (x_name, relation, y_name)={ edge_index=[2, 27106] },
  (y_name, rev_relation, x_name)={ edge_index=[2, 27106] }
)


In [9]:
##############################  Defining Edge-level Training Splits ##############################

# For this, we first split the set of edges into
# training (80%), validation (10%), and testing edges (10%).
# Across the training edges, we use 70% of edges for message passing (computing the embeddings of the nodes),

# and 30% of edges for supervision (used as the label data to train the model)
# The model learns to predict the existence (or absence) of these specific edges based on the node embeddings generated from the message-passing phase.

# We further want to generate fixed negative edges for evaluation with a ratio of 2:1.
# Negative edges during training will be generated on-the-fly, so we don't want to
# add them to the graph right away.
# Overall, we can leverage the `RandomLinkSplit()` transform for this from PyG:

transform = T.RandomLinkSplit(
    num_val=0.1,   # The percentage of edges in the validation set
    num_test=0.1,  # The percentage of edges in the test set
    disjoint_train_ratio=0.3,  # The percentage of edges used for supervision in the training set
    neg_sampling_ratio=2,      # The ratio of negative to positive edges
    add_negative_train_samples=False,  # False here means we will generate negative edges during training
    edge_types=(triplet_names),
    rev_edge_types=(triplet_names_rev),
)

train_data, val_data, test_data = transform(data)
print("Training data:")
print("==============")
print(train_data)
print()
print("Validation data:")
print("================")
print(val_data)


Training data:
HeteroData(
  x_name={
    node_id=[961],
    x=[961, 480],
  },
  y_name={
    node_id=[4362],
    x=[4362, 480],
  },
  (x_name, relation, y_name)={
    edge_index=[2, 15181],
    edge_label=[6505],
    edge_label_index=[2, 6505],
  },
  (y_name, rev_relation, x_name)={ edge_index=[2, 15181] }
)

Validation data:
HeteroData(
  x_name={
    node_id=[961],
    x=[961, 480],
  },
  y_name={
    node_id=[4362],
    x=[4362, 480],
  },
  (x_name, relation, y_name)={
    edge_index=[2, 21686],
    edge_label=[8130],
    edge_label_index=[2, 8130],
  },
  (y_name, rev_relation, x_name)={ edge_index=[2, 21686] }
)


In [256]:
import numpy as np
# check values
test_data[triplet_names].edge_label_index[ 0 , :]

tensor([447, 285, 166,  ..., 717, 815, 170])

In [139]:
torch.max( (test_data[triplet_names].edge_label_index[ 0 , :]) )

tensor(960)

In [145]:
torch.max( (test_data[triplet_names].edge_label_index[ 1 , :]) )

tensor(4360)

In [150]:
torch.max( test_data[triplet_names].edge_index[ 1 , :] )

tensor(4361)

In [148]:
torch.unique(test_data[triplet_names].edge_index, return_counts=True)

(tensor([   0,    1,    2,  ..., 4359, 4360, 4361]),
 tensor([316, 209, 190,  ...,   1,   1,   1]))

In [149]:
torch.unique(test_data[triplet_names].edge_label_index, return_counts=True)

(tensor([   0,    1,    2,  ..., 4358, 4359, 4360]),
 tensor([48, 32, 22,  ...,  4,  1,  1]))

In [98]:
# ground truth.
test_data[triplet_names].edge_label
#torch.unique(test_data[triplet_names].edge_label, return_counts=True)

tensor([1., 1., 1.,  ..., 0., 0., 0.])

In [99]:
len(test_data[triplet_names].edge_label)

8130

In [118]:
# ground truth.
edge_index_x_to_y[:, edge_index_x_to_y[1, :] == 33 ]

tensor([[ 35,  36,  37,  38, 146, 210, 269,  60, 549, 277, 177,  27, 352,   5,
         214,  41, 107,  89, 273, 410, 161, 425,  29, 165, 192,  39,   0,  87,
          78, 696, 266],
        [ 33,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33,
          33,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33,
          33,  33,  33]])

In [121]:
# What is this ??? positive link are not present
test_edge_index = test_data[triplet_names].edge_index
test_edge_index[:, test_edge_index[1, :] == 33 ]

tensor([[277, 273, 107, 549, 146,  29,  89, 269, 161,  36,  41, 352,  37,  38,
         425, 210,   0,   5, 165, 192,  87,  78, 177,  60,  27, 410,  39, 266],
        [ 33,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33,
          33,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33]])

In [10]:
############################## Defining Mini-batch Loaders ##############################

# In the first hop, we sample at most 20 neighbors.
# In the second hop, we sample at most 10 neighbors.
# In addition, during training, we want to sample negative edges on-the-fly with
# a ratio of 2:1.
# We can make use of the `loader.LinkNeighborLoader` from PyG:
from torch_geometric.loader import LinkNeighborLoader

# Define seed edges:
edge_label_index = train_data[triplet_names].edge_label_index
edge_label = train_data[triplet_names].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[20, 10],
    neg_sampling_ratio=2,
    edge_label_index=(triplet_names, edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

Sampled mini-batch:
HeteroData(
  x_name={
    node_id=[806],
    x=[806, 480],
    n_id=[806],
  },
  y_name={
    node_id=[2116],
    x=[2116, 480],
    n_id=[2116],
  },
  (x_name, relation, y_name)={
    edge_index=[2, 8209],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[8209],
    input_id=[128],
  },
  (y_name, rev_relation, x_name)={
    edge_index=[2, 5115],
    e_id=[5115],
  }
)


/usr/local/lib/python3.10/dist-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


In [11]:
################################# Creating a Heterogeneous Link-level GNN #################################

from torch_geometric.nn import SAGEConv, to_hetero
from torch import Tensor

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()

        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        # Define a 2-layer GNN computation graph. Neighborhood depth K is 2.
        # Use a single `ReLU` non-linearity in-between. why doing this ?
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        return x

# Our final classifier applies the dot-product between source and destination
# node embeddings to derive edge-level predictions:
class Classifier(torch.nn.Module):
    def forward(self, x_source: Tensor, x_target: Tensor, edge_label_index: Tensor) -> Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_source = x_source[edge_label_index[0]]
        edge_feat_target = x_target[edge_label_index[1]]

        # Apply dot-product to get a prediction per supervision edge:
        return (edge_feat_source * edge_feat_target).sum(dim=-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # Since the dataset does not come with rich features, we also learn two
        # embedding matrices for source and target nodes:
        self.source_lin = torch.nn.Linear(data["x_name"].x.shape[1], hidden_channels)
        self.target_lin = torch.nn.Linear(data["y_name"].x.shape[1], hidden_channels)

        self.source_emb = torch.nn.Embedding(data["x_name"].num_nodes, hidden_channels)
        self.target_emb = torch.nn.Embedding(data["y_name"].num_nodes, hidden_channels)

        # Instantiate homogeneous GNN:
        self.gnn = GNN(hidden_channels)
        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())
        # Instantiate edge-level classifier:
        self.classifier = Classifier()

    def forward(self, data: HeteroData) -> Tensor:
        x_dict = {
          #"x_name": self.source_emb(data["x_name"].node_id),
          "x_name": self.source_lin(data["x_name"].x)  + self.source_emb(data["x_name"].node_id),
          #"y_name": self.target_emb(data["y_name"].node_id),
          "y_name": self.target_lin(data["y_name"].x) + self.target_emb(data["y_name"].node_id),
        }
        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
        x_dict = self.gnn(x_dict, data.edge_index_dict)

        # Apply final classifier to derive edge-level predictions:
        pred = self.classifier(
            x_dict["x_name"],
            x_dict["y_name"],
            data[triplet_names].edge_label_index,
        )

        return pred




In [12]:
################################ Training a Heterogeneous Link-level GNN ################################

import tqdm
import torch.nn.functional as F

model = Model(hidden_channels=64)
print(model)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


for epoch in range(1, 10):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()

        sampled_data = sampled_data.to(device)  # moving to device
        pred = model(sampled_data)  # forward pass
        # `F.binary_cross_entropy_with_logits(pred, ground_truth)`
        ground_truth = sampled_data[(triplet_names)]['edge_label']
        loss = F.binary_cross_entropy_with_logits(pred, ground_truth)  # BCE loss

        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

# at this point the model is already trained based on training set.
torch.save(model, 'model/model_CollecTRI.pt')


Model(
  (source_lin): Linear(in_features=480, out_features=64, bias=True)
  (target_lin): Linear(in_features=480, out_features=64, bias=True)
  (source_emb): Embedding(961, 64)
  (target_emb): Embedding(4362, 64)
  (gnn): GraphModule(
    (conv1): ModuleDict(
      (x_name__relation__y_name): SAGEConv(64, 64, aggr=mean)
      (y_name__rev_relation__x_name): SAGEConv(64, 64, aggr=mean)
    )
    (conv2): ModuleDict(
      (x_name__relation__y_name): SAGEConv(64, 64, aggr=mean)
      (y_name__rev_relation__x_name): SAGEConv(64, 64, aggr=mean)
    )
  )
  (classifier): Classifier()
)
Device: 'cuda'


100%|██████████| 51/51 [00:02<00:00, 20.05it/s]


Epoch: 001, Loss: 0.6569


100%|██████████| 51/51 [00:01<00:00, 42.98it/s]


Epoch: 002, Loss: 0.4871


...

Epoch: 007, Loss: 0.3302


100%|██████████| 51/51 [00:01<00:00, 40.00it/s]


Epoch: 008, Loss: 0.3204


100%|██████████| 51/51 [00:01<00:00, 41.56it/s]


Epoch: 009, Loss: 0.3136


In [13]:
################################ Evaluating a Heterogeneous Link-level GNN ################################

# load model
model = torch.load('model/model_CollecTRI.pt')

# Define the validation seed edges:
edge_label_index = val_data[triplet_names].edge_label_index
edge_label = val_data[triplet_names].edge_label

val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[20, 10],
    edge_label_index=((triplet_names), edge_label_index),
    edge_label=edge_label,
    batch_size=3 * 128,
    shuffle=False,
)

sampled_data = next(iter(val_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)


from sklearn.metrics import roc_auc_score

preds = []
ground_truths = []
for sampled_data in tqdm.tqdm(val_loader):
    with torch.no_grad():

        pred = model(sampled_data.to(device))
        preds.append(pred)

        ground_truth = sampled_data[(triplet_names)]['edge_label']
        ground_truths.append(ground_truth)

pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
auc = roc_auc_score(ground_truth, pred)
print(f"Validation AUC: {auc:.4f}")

Sampled mini-batch:
HeteroData(
  x_name={
    node_id=[814],
    x=[814, 480],
    n_id=[814],
  },
  y_name={
    node_id=[2304],
    x=[2304, 480],
    n_id=[2304],
  },
  (x_name, relation, y_name)={
    edge_index=[2, 11227],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[11227],
    input_id=[384],
  },
  (y_name, rev_relation, x_name)={
    edge_index=[2, 6373],
    e_id=[6373],
  }
)


100%|██████████| 22/22 [00:00<00:00, 42.10it/s]

Validation AUC: 0.8647


In [14]:
val_data

HeteroData(
  x_name={
    node_id=[961],
    x=[961, 480],
  },
  y_name={
    node_id=[4362],
    x=[4362, 480],
  },
  (x_name, relation, y_name)={
    edge_index=[2, 21686],
    edge_label=[8130],
    edge_label_index=[2, 8130],
  },
  (y_name, rev_relation, x_name)={ edge_index=[2, 21686] }
)

In [17]:
np.max( df_x_name.tolist() )
np.max( df_y_name.tolist() )

4361

In [224]:
# assess performance on test set

H_data = val_data

pred = model(H_data.to(device)).tolist()
ground_truth = H_data[(triplet_names)]['edge_label'].tolist()

auc = roc_auc_score(ground_truth, pred)
print(f"Validation AUC: {auc:.4f}")

Validation AUC: 0.8604


In [243]:
H_data

HeteroData(
  x_name={
    node_id=[961],
    x=[961, 480],
  },
  y_name={
    node_id=[4362],
    x=[4362, 480],
  },
  (x_name, relation, y_name)={
    edge_index=[2, 21686],
    edge_label=0,
    edge_label_index=[2, 8130],
  },
  (y_name, rev_relation, x_name)={ edge_index=[2, 21686] }
)

In [33]:
import copy
H_data = copy.deepcopy(val_data)
H_data[triplet_names].edge_label = 0
pred = model(H_data.to(device)).tolist()
pred[0:10]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [26]:
H_data[triplet_names].edge_label_index

tensor([[  51,  706,  157,  ...,  144,  919,  418],
        [4361,  695, 3086,  ..., 4115,  923, 1259]], device='cuda:0')

In [32]:
H_data[triplet_names].edge_label_index[1,0] = 4361

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [31]:
# changed the edges for prediction
pred = model(H_data.to(device)).tolist()
pred[0:10]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
